# Load API Keys

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# Tracing

In [2]:
from langfuse.openai import AsyncOpenAI

# Setup LLM Call Helpers

In [3]:
client = AsyncOpenAI()

In [4]:
GPT4O_MINI = "o4-mini-2025-04-16"

In [5]:
def _msg(role, content):
    return {'role': role, 'content': content}

def system(content):
    return _msg('system', content)

def user(content):
    return _msg('user', content)

def assistant(content):
    return _msg('assistant', content)

# Cache to disk

In [6]:
from diskcache import Cache

In [7]:
cache = Cache(directory=".cache_course")

In [8]:
import asyncio

In [9]:
async def set_async(key, val, **kwargs):
    return await asyncio.to_thread(cache.set, key, val, **kwargs)

async def get_async(key, default=None, **kwargs):
    return await asyncio.to_thread(cache.get, key, default, **kwargs)

# Implementing Cached, Retried and Traced Structures Outputs Completion

In [10]:
from pydantic import BaseModel

In [11]:
import json
from hashlib import md5

def make_cache_key(key_name, **kwargs):
    kwargs_string = json.dumps(kwargs, sort_keys=True)
    kwargs_hash = md5(kwargs_string.encode('utf-8')).hexdigest()
    cache_key = f"{key_name}__{kwargs_hash}"
    
    return cache_key

In [12]:
def _make_key_for_cached_chat_completion_parsed_with_retry(
    *,
    model,
    messages,
    response_format: BaseModel,
    **kwargs,
):
    return make_cache_key(
        "openai_parsed_chat",
        model=model,
        messages=messages,
        response_format=response_format.model_json_schema(),
        **kwargs
    )

In [15]:
#from openai.types.responses import ParsedResponse
from openai.types.chat import ParsedChatCompletion
from functools import wraps
from openai import APITimeoutError, RateLimitError
# from pydantic import BaseModel
from typing_extensions import TypeVar
import backoff

ResponseFormatT = TypeVar("ResponseFormatT", bound=BaseModel)

CACHE_MISS_SENTINEL = object()

@wraps(client.chat.completions.parse)
async def cached_chat_completion_parsed_with_retry(
    *,
    model,
    messages,
    response_format: ResponseFormatT,
    **kwargs,
) -> ParsedChatCompletion[ResponseFormatT]:
    # CREATE CACHE KEY
    cache_key = _make_key_for_cached_chat_completion_parsed_with_retry(
        model=model,
        messages=messages,
        response_format=response_format,
        **kwargs
    )
    cached_value = await get_async(cache_key, default=CACHE_MISS_SENTINEL)
    
    # CACHE MISS
    if cached_value is CACHE_MISS_SENTINEL:
        @backoff.on_exception(
            backoff.expo,
            (APITimeoutError, RateLimitError)
        )
        async def do_call():
            return await client.chat.completions.parse(
                model=model,
                messages=messages,
                response_format=response_format,
                **kwargs
            )
        completion = await do_call()
        await set_async(cache_key, completion.model_dump_json())
        return completion
    # CACHE HIT
    else:
        completion = ParsedChatCompletion.model_validate(json.loads(cached_value))
        for choice in completion.choices:
            if not choice.message.refusal:
                choice.message.parsed = response_format.model_validate(
                    choice.message.parsed
                )
        return completion

In [16]:
class CalendarEvent(BaseModel):                 #####################################
    name: str                                   ############# SCHEMA ################
    date: str                                   #####################################
    participants: list[str]                     #####################################

completion = await cached_chat_completion_parsed_with_retry(
    model=GPT4O_MINI,
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Alice and Bob are going to a science fair on Friday.",
        },
    ],
    response_format=CalendarEvent,
    # max_output_tokens=5  ### WILL NOT WORK because not enough tokens to output the full json string
)

print(completion.choices[0].message.parsed)

name='Science Fair' date='Friday' participants=['Alice', 'Bob']


In [18]:
completion

ParsedChatCompletion(id='chatcmpl-C5u7UShKq6BgMJhZM5O896DOUEeba', choices=[ParsedChoice(finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage(content='{"name":"Science Fair","date":"Friday","participants":["Alice","Bob"]}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None, parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])))], created=1755523400, model='o4-mini-2025-04-16', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=167, prompt_tokens=90, total_tokens=257, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=128, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [ ]:
event

# Demo

In [ ]:
# instead of
# client.chat.completions.create
# use:
from pydantic import BaseModel

class CalendarEvent(BaseModel):                 #####################################
    name: str                                   ############# SCHEMA ################
    date: str                                   #####################################
    participants: list[str]                     #####################################

response = await client.responses.parse(
    model=GPT4O_MINI,
    input=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Alice and Bob are going to a science fair on Friday.",
        },
    ],
    text_format=CalendarEvent,
    # max_output_tokens=5  ### WILL NOT WORK because not enough tokens to output the full json string
)

event = response.output_parsed

In [ ]:
response

In [ ]:
event.model_dump_json()

In [ ]:
import json

response_as_dict = json.loads(event.model_dump_json())
print(response_as_dict)

In [ ]:
event = CalendarEvent.model_validate(response_as_dict)

In [ ]:
event.name

In [ ]:
CalendarEvent

In [ ]:
event